In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import HTML
import matplotlib.animation as animation

In [ ]:
%matplotlib inline

## Generating Data

In [ ]:
seed = 6343
np.random.seed(seed)

In [ ]:
n = 15
m = 15

In [ ]:
farm_clusters = (np.random.random((n, 2)) * 10).round(decimals=1)
ranges = np.random.random((n, 2)) * 0.3

farm_pixels = []
for i in range(n):
    xs = farm_clusters[i][0] - ranges[i][0]
    xe = farm_clusters[i][0] + ranges[i][0]

    ys = farm_clusters[i][1] - ranges[i][1]
    ye = farm_clusters[i][1] + ranges[i][1]

    for x in np.arange(xs, xe + 0.1, 0.1):
        for y in np.arange(ys, ye + 0.1, 0.1):
            farm_pixels.append([x, y])

farm_pixels = np.array(farm_pixels).round(decimals=1)
farm_pixels = farm_pixels[~((farm_pixels[:, 0] < 0) | (farm_pixels[:, 1] < 0))]

wells = (np.random.random((m, 2)) * 10).round(decimals=1)

In [ ]:
seasonal_factor = [-1, -2, 3, 0]

In [ ]:
base_water_level = -np.random.randint(0, n)
distance_weight = np.random.random()
farm_factor = np.random.random()

In [ ]:
def get_observations(
    wells, farms, farm_factor, distance_weight, base_water_level, seasonal_factor
):
    def pairwise_distances(x, y=None):
        x_norm = (x ** 2).sum(1)[:, None]
        y_norm = (y ** 2).sum(1)[None, :]

        dist = x_norm + y_norm - 2.0 * np.matmul(x, y.T)

        return dist

    def get_observations_at_timestep(wells, farms, base):
        obs = (
            base
            - np.sum(np.exp(-pairwise_distances(wells, farms) / distance_weight), 1)
            * farm_factor
        )
        obs += np.random.randn(*obs.shape) * 0.5

        return obs

    base_water_levels = list()
    for i in range(40):
        base_water_level = min(
            base_water_level
            + seasonal_factor[i % 4]
            + np.random.randn() / 5,
            0,
        )

        base_water_levels.append(base_water_level)
        
    parameters = {"mu_{}".format(i): base_water_levels[i] for i in range(40)}
    
    parameters["delta"] = farm_factor
    parameters["theta_f"] = distance_weight

    return parameters, [
        get_observations_at_timestep(wells, farms, base_water_levels[i])
        for i in range(40)
    ]

In [ ]:
parameters, observations = get_observations(
    wells, farm_pixels, farm_factor, distance_weight, base_water_level, seasonal_factor
)

with open("data/sample_data2.csv", "w") as f:
    f.write("timestep,type,latitude,longitude,observation\n")

    f.writelines(["0,farm,{},{},1\n".format(x, y) for x, y in farm_pixels])
    
    for t, observations_ in enumerate(observations):
        f.writelines(
            ["{},well,{},{},{}\n".format(t, x, y, o) for (x, y), o in zip(wells, observations_)]
        )

In [ ]:
plt.clf()
fig = plt.figure(figsize=(10, 10), dpi=100)

plt.ion()

plt.scatter(farm_pixels[:, 0], farm_pixels[:, 1], marker="s", s=5, color="lightgreen")

scat = plt.scatter(wells[:, 0], wells[:, 1], marker="s", s=5, c=[(0, 0, 0, 1)] * len(wells))
label = plt.text(0, 0, '', fontsize=7)

colors = []
for observations_ in observations:
    min_v = min(observations_)
    max_v = max(observations_)
#     colors.append([max((x - min_v) / (max_v - min_v), 0.1) for x in observations_])
    colors.append([1 - abs(x) / 25 for x in observations_])
    
colors = np.array(colors)

def update_plot(i, scat):
    scat.set_array(colors[i])
    label.set_text(["Sp", "Su", "Fa", "Wi"][i % 4])
    return scat,

ani = animation.FuncAnimation(fig, update_plot, frames=range(len(observations)), fargs=(scat,), interval=1000)

plt.gray()
plt.close();

In [ ]:
HTML(ani.to_html5_video())

In [ ]:
with open("data/kr-samples.json", "r") as f:
    kr_samples = {k: np.array(v) for k, v in json.load(f).items()}

In [ ]:
with open("data/gp-samples.json", "r") as f:
    gp_samples = {k: np.array(v) for k, v in json.load(f).items()}

In [ ]:
keys = set(kr_samples.keys()).intersection(parameters.keys()).intersection(gp_samples.keys())

In [ ]:
for k in sorted(list(keys)):
    print("{0:7} | {1:6.3f} {2:7.3f} {3:7.3f}".format(k, parameters[k], kr_samples[k].mean(0), gp_samples[k].mean(0)))